In this notebook, we will be downloading data from the [**Danish Climate Outlook 2023**](https://ens.dk/service/fremskrivninger-analyser-modeller/klimastatus-og-fremskrivning-2023) that is produced by the Danish Energy Agency. The Danish Energy Agency uses a linear programming model of the European electricity system called [**RAMSES**](https://ens.dk/sites/ens.dk/files/Analyser/ramses_energisystemmodel.pdf) as part of the producing the climate outlook. The input data for Ramses is based on an inventory of all Danish electricity and district heat generating plants in Denmark as well as a set of representative plants in the rest of Europe. The Ramses data is confidential, but the Danish Energy Agency does make available a list of representative plants for each area/country. 

We will be using this data on representative plants. It is made available on their [**website**](https://ens.dk/service/fremskrivninger-analyser-modeller/klimastatus-og-fremskrivning-2023) under the heading **"Dataark for resultater"**, but can also be directly downloaded here: [**"KF23 dataark – El og fjernvarme"**](https://ens.dk/sites/ens.dk/files/Basisfremskrivning/kf23_el_og_fjernvarme.xlsx).  Unfortunately, the data is only available in Danish but it is not difficult to translate using Google translate. We will be using:

Import standard packages:

In [1]:
import pandas as pd,os, numpy as np

Let's specify an output folder:

In [2]:
direc = os.getcwd()
data_dir = os.path.join(direc,'CleanedData')

## 1 Settings

In [3]:
year = 2022

Electricity areas:

In [4]:
g_E = {'DK1':'DK1','DK2':'DK2'}

Heating areas:

In [5]:
g_H = {
    'Centrale områder':'Central', # Large plants located at central nodes in the grid (often urban areas)
    'Større decentrale områder':'LargeDecentral', # large plant distributed close to actual consumption (distribution generation)
    'Mindre decentrale områder':'SmallDecentral', # small plants distributed very close to actual consumption (distributed generation)
}

Technology types:

In [6]:
tech = {
    'Industriel overskudsel':'IndustryE', # Industrial surplus electricity
    'Kondens':'CD', 
    'Havvind':'WS', 
    'Hydro':'ROR', # Run-of-river hydro (intermittent without storage)
    'Kedel':'BH',
    'Kraftvarme':'BP', # Assume all CHP plants are Back Pressure. 
    'Industrivarme':'IndustryH', # Industrial surplus heat
    'PtX_Brint': 'EP', # Electrolyzer Plant 
    'Elpatron':'IH', # Electric immersion water heater 
    'Solvarme':'SH',
    'Varmepumper':'HPstandard', 
    'Varmepumper(overskudsvarme)':'HPsurplusheat', 
    'Geotermi':'GT',
    'Landvind':'WL', 
    'Solceller':'PV',
}

Mapping between tech and model tech:

In [7]:
tech2modelTech = {tech:'standard_E' for tech in ['IndustryE','CD','WS','ROR','WL','PV']}
tech2modelTech.update({tech:'standard_H' for tech in ['BH','IndustryH','SH','GT']})
tech2modelTech.update({'BP':'BP','HPstandard':'HP','HPsurplusheat':'HP','IH':'HP','EP':'HP'})

Fuel types:

In [8]:
BFt = {
    'Biogas':'Biogas',
    'Havvind':np.nan,
    'Hydro':np.nan,
    'Naturgas':'Natgas',
    'Olie':'Oil',
    'Affald':'Waste',
    'Biomasse':'Biomass',
    'Elkedler':np.nan,
    'Kul':'Coal',
    'Solvarme':np.nan,
    'Varmepumper':np.nan,
    'Varmepumper(overskudsvarme)':np.nan,
    'Geotermi':np.nan,
    'Landvind':np.nan,
    'Solceller':np.nan,
    'Industrivarme':np.nan
}

## 2 Clean Plant Data

Get raw plant data:

In [9]:
df_plant = pd.read_excel(os.path.join(os.getcwd(),'RawData','ClimateOutlook2023_PlantData_and_TTC.xlsx'),sheet_name='Rådata_prod').drop(columns='version').rename(columns={
    'year':'Year',
    'ElArea':'g_E',
    'HeatArea_Category':'g_H',
    'Teknologitype':'TechnologyType',
    'Brændselstype':'BFt',
    'Elkapacitet_MW':'GeneratingCapacity_E',
    'Varmekapacitet_MW':'GeneratingCapacity_H',
    'Elproduktion_TWh':'Generation_E',
    'Varmeproduktion_TWh':'Generation_H',
    'Brændselsforbrug_TWh':'FuelConsumption',
})

---

In [10]:
df_plant.head()

,Year,g_E,g_H,BFt,TechnologyType,GeneratingCapacity_E,GeneratingCapacity_H,Generation_E,Generation_H,FuelConsumption
0,2022,AT,NaN,Biomasse,Kraftvarme,482.000,0.0,2.715,0.0,3.824
1,2022,AT,NaN,Hydro,Hydro,8728.000,0.0,27.556,0.0,25.188
2,2022,AT,NaN,Landvind,Landvind,3500.000,0.0,7.187,0.0,7.187
3,2022,AT,NaN,Naturgas,Industriel overskudsel,1057.000,0.0,5.259,0.0,11.687
4,2022,AT,NaN,Naturgas,Kondens,4348.999,0.0,1.693,0.0,2.930


---

Subset year:

In [11]:
df_plant = df_plant[df_plant['Year']==year]

Subset and aggregate electricity area:

In [12]:
df_plant = df_plant[df_plant['g_E'].isin(g_E.keys())].replace({'g_E':g_E})

Subset and aggregate district heat area:

In [13]:
df_plant = df_plant[df_plant['g_H'].isin([x for x in g_H.keys()] + [np.nan])].replace({'g_H':g_H})

Aggregate fuel types:

In [14]:
df_plant['BFt'] = df_plant['BFt'].replace(BFt)

Deal with PtX because it is split over multiple cells:

In [15]:
idx = df_plant['TechnologyType']=='PtX_Brint'
df_plant.loc[idx,'BFt'] = np.nan

Aggregate technology types:

In [16]:
df_plant['tech'] = ['_'.join([y,x]) if isinstance(x,str) else y for x,y in zip(df_plant['BFt'],df_plant['TechnologyType'].replace(tech))]

Add model technology:

In [17]:
df_plant['modelTech'] = df_plant['TechnologyType'].replace(tech).replace(tech2modelTech)

Correct some purely electricity producing plants that are part of heating areas:

In [18]:
idx = (~df_plant['g_H'].isna()) & (df_plant['GeneratingCapacity_H']==0) & (df_plant['tech']!='EP')
df_plant.loc[idx,'g_H'] = np.nan

Aggregate plants:

In [19]:
g_cols = ['Year','g_E','g_H','BFt','tech','modelTech']
num_cols = ['GeneratingCapacity_E','GeneratingCapacity_H','Generation_E','Generation_H','FuelConsumption']
df_plant[num_cols] = df_plant[num_cols].astype(float)
df_plant = df_plant.groupby(g_cols,dropna=False)[num_cols].agg('sum').reset_index()

Calculate inverse fuel efficiencies for standard thermal plants:

In [20]:
df_plant['FuelMix'] = np.nan
idx = (df_plant['modelTech'].str.find('standard_')!=-1) & (~df_plant['BFt'].isna())
df_plant.loc[idx,'FuelMix'] = df_plant.loc[idx,'FuelConsumption']/(df_plant.loc[idx,'Generation_E']+df_plant.loc[idx,'Generation_H'])

Calculate inverse fuel efficiens for back pressure plants:

In [21]:
idx = df_plant['modelTech']=='BP'
df_plant.loc[idx,'FuelMix'] = df_plant.loc[idx,'FuelConsumption']/df_plant.loc[idx,'Generation_E']

Calculate electricity-to-heat ratio of back pressure, heat pumps and ptx plants:

In [22]:
df_plant['E2H'] = np.nan
idx = df_plant['modelTech'].isin(['BP','HP'])
df_plant.loc[idx,'E2H'] = df_plant.loc[idx,'GeneratingCapacity_E']/df_plant.loc[idx,'GeneratingCapacity_H']

Calculate electricity to hydrogen for ptx plants:

In [23]:
df_plant['E2HH'] = np.nan
idx = df_plant['tech']=='EP'
df_plant.loc[idx,'E2HH'] = df_plant.loc[idx,'Generation_E']/df_plant.loc[idx,'FuelConsumption']

Correct capacities:

In [24]:
df_plant = df_plant.replace({'GeneratingCapacity_E':{0:np.nan},'GeneratingCapacity_H':{0:np.nan}})
df_plant.loc[df_plant['modelTech']=='BP','GeneratingCapacity_H'] = np.nan
df_plant.loc[df_plant['modelTech']=='HP','GeneratingCapacity_E'] = np.nan

Subset database:

In [25]:
df_plant = df_plant[g_cols+['GeneratingCapacity_E','GeneratingCapacity_H','FuelMix','E2H','E2HH','Generation_E','Generation_H']]

Make plant id:

In [26]:
df_plant['id'] = ['id_'+'_'.join([str(x),str(y),str(z)]) for x,y,z in zip(df_plant['g_E'],df_plant['g_H'],df_plant['tech'])]

Adjust g_H label:

In [27]:
df_plant['g_H'] = ['_'.join([g_E,g_H]) if isinstance(g_H,str) else g_H for g_E,g_H in zip(df_plant['g_E'],df_plant['g_H'])]

Save database:

In [28]:
file_path = os.path.join(data_dir,'PlantData_DK_'+str(year))
df_plant.to_pickle(file_path)

---

In [29]:
obj = pd.read_pickle(file_path)
obj.head()

,Year,g_E,g_H,BFt,tech,modelTech,GeneratingCapacity_E,GeneratingCapacity_H,FuelMix,E2H,E2HH,Generation_E,Generation_H,id
0,2022,DK1,DK1_Central,Biogas,BP_Biogas,BP,9.412,NaN,2.525641,0.895869,NaN,0.078,0.085,id_DK1_Central_BP_Biogas
1,2022,DK1,DK1_Central,Biogas,IndustryH_Biogas,standard_H,NaN,8.414,0.565217,NaN,NaN,0.000,0.023,id_DK1_Central_IndustryH_Biogas
2,2022,DK1,DK1_Central,Biomass,BH_Biomass,standard_H,NaN,206.306,0.981132,NaN,NaN,0.000,0.106,id_DK1_Central_BH_Biomass
3,2022,DK1,DK1_Central,Biomass,BP_Biomass,BP,663.909,NaN,3.415162,0.507757,NaN,3.324,5.473,id_DK1_Central_BP_Biomass
4,2022,DK1,DK1_Central,Coal,BP_Coal,BP,970.006,NaN,2.645989,0.771789,NaN,5.497,2.702,id_DK1_Central_BP_Coal


## 2 Clean TTC Data

Get raw data:

In [29]:
df_ttc = pd.read_excel(os.path.join(os.getcwd(),'RawData','ClimateOutlook2023_PlantData_and_TTC.xlsx'),sheet_name='Rådata_NTC').drop(columns='version').rename(columns={'From':'g_E','To':'g_EE','NTC_MW':'TTC','year':'Year'})

Subset year:

In [30]:
df_ttc = df_ttc[df_ttc['Year']==year]

Subset to DK:

In [31]:
df_ttc = df_ttc[(df_ttc['g_E'].isin(g_E.keys())) | (df_ttc['g_EE'].isin(g_E.keys()))].replace({'g_E':g_E,'g_EE':g_E})

Drop DK to DK if aggregated:

In [32]:
df_ttc = df_ttc[df_ttc['g_E']!=df_ttc['g_EE']].reset_index(drop=True)

Shift columns to make export and import:

In [33]:
# idx = (df_ttc['g_E'].str.find('DK')!=-1) 
# idx_DK = (idx) & (df_ttc['g_EE'].str.find('DK')!=-1)
# df_ttc['ExportCapacity'] = 0
# df_ttc.loc[idx,'ExportCapacity'] = df_ttc.loc[idx,'TTC']
# df_ttc['ImportCapacity'] = 0
# df_ttc.loc[~idx,'ImportCapacity'] = df_ttc.loc[~idx,'TTC']
# df_ttc.loc[idx_DK,'ImportCapacity'] = df_ttc.loc[idx_DK,'TTC']
# g_EE = df_ttc.loc[(~idx) & (~idx_DK),'g_EE']
# df_ttc.loc[(~idx) & (~idx_DK),'g_EE'] = df_ttc.loc[(~idx) & (~idx_DK),'g_E']
# df_ttc.loc[(~idx) & (~idx_DK),'g_E'] = g_EE

Aggregate:

In [34]:
df_ttc = df_ttc.groupby(['Year','g_E','g_EE']).agg('sum').reset_index()

Save database:

In [35]:
file_path = os.path.join(data_dir,'TTC_DK_'+str(year))
df_ttc.to_pickle(file_path)